In [80]:
# setup work space
import arcpy
import os
import glob
import tempfile
# Set environment settings
arcpy.env.workspace = "C:/Workspace/VTWetlands/VTWetlands/"
wd = arcpy.env.workspace + "/"
print(os.listdir(wd))
print(arcpy.env.workspace)
outfolder = arcpy.env.workspace + "/outputs/"
print(outfolder)

# random number generator and seed 
arcpy.env.randomGenerator = "99 ACM599"

# global parameters
nstudents = 71

['.backups', '.ipynb_checkpoints', 'AssignRandomPoints2KMLs.ipynb', 'ImportLog', 'Index', 'outputs', 'sites', 'VSWI_Wetlands_Advisory_Layer', 'VSWI_Wetlands_Advisory_Layer.zip', 'VSWI_Wetlands_Class_Layer', 'VSWI_Wetlands_Class_Layer.zip', 'VTWetlands.aprx', 'VTWetlands.aprx.xml', 'VTWetlands.gdb', 'VTWetlands.tbx', 'Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017', 'Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017.zip']
C:/Workspace/VTWetlands/VTWetlands
C:/Workspace/VTWetlands/VTWetlands/outputs/


In [82]:
# Generate random points inside dissolved polygon

# dissolve polygons of constraining feature
infile = "Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017/Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017.shp"
inpath = wd + infile
print(inpath)
outfeatures = "Dissolved.shp"
outpath = outfolder + outfeatures
# check if file exists and delete before running script
arcpy.Dissolve_management (inpath, outpath)

C:/Workspace/VTWetlands/VTWetlands/Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017/Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017.shp


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000725: Output Feature Class: Dataset C:\Workspace\VTWetlands\VTWetlands\outputs\Dissolved.shp already exists.
Failed to execute (Dissolve).


In [88]:
# create 1 random points inside each polygon
outname = "randompoints.shp"
npoints = nstudents * 10
mindistance = 100
arcpy.management.CreateRandomPoints(out_path=outfolder, 
                                    out_name=outname, 
                                    constraining_feature_class=inpath,             
                                    number_of_points_or_field=1, 
                                    minimum_allowed_distance= mindistance)

# create n random points inside one dissovled polygon
outname = "dissolved_randompoints.shp"
npoints = nstudents * 10
mindistance = 100
arcpy.management.CreateRandomPoints(out_path=outfolder, 
                                    out_name=outname, 
                                    constraining_feature_class=outpath,             
                                    number_of_points_or_field=npoints, 
                                    minimum_allowed_distance= mindistance)

<Result 'C:/Workspace/VTWetlands/VTWetlands/outputs\\dissolved_randompoints.shp'>

In [110]:
# outfile names
outnames = ['vtss_hygr',
'lclu16_wet',
'lclu16_shr',
'lclu16_agr',
'rcpp_score',
'vswi_class',
'vswi_advis']

# value_fields
invarnames = [
    'HyGrNum',
    'Class_name',
    'Class_name',
    'Class_name',
    'RCPP_SCORE',
    'CLASS',
    'CLASS']

pathnames = [r'D:\GeoData\VT_Data_-_NRCS_Soil_Survey_Units-shp\VT_Data_-_NRCS_Soil_Survey_Units.shp',
r'D:\GeoData\LandLandcov_Wetlands2016\LandLandcov_Wetlands2016.gdb\Land_Wetlands2016_poly',
r'D:\GeoData\LandLandcov_Shrublands2016\LandLandcov_Shrublands2016.gdb\Land_Shrublands2016_poly',
r'D:\GeoData\LandLandcov_Agriculture2016\LandLandcov_Agriculture2016.gdb\Land_Agriculture2016_poly',
r'D:\GeoData\VSWI\Wetland_Restoration_Model_Site_Prioritization__Lake_Champlain_2017_-shp\Wetland_Restoration_Model_Site_Prioritization__Lake_Champlain_2017_.shp',
r'D:\GeoData\VSWI\VSWI_Wetlands_Class_Layer-shp\VSWI_Wetlands_Class_Layer.shp',
r'D:\GeoData\VSWI\VSWI_Wetlands_Advisory_Layer-shp\VSWI_Wetlands_Advisory_Layer.shp']

In [112]:
for i in range(len(pathnames)):
    print(i)
    in_features=pathnames[i]
    value_field=invarnames[i]
    out_rasterdataset=outfolder+outnames[i]+".tif"
    print(out_rasterdataset)
    try:
        arcpy.conversion.PolygonToRaster(in_features,
                                     value_field,
                                     out_rasterdataset)
                                     #build_rat = "BUILD")
        print("task complete",os.path.exists(out_rasterdataset))
    except:
        print("task failed\n outpath exists:",os.path.exists(out_rasterdataset))

0
C:/Workspace/VTWetlands/VTWetlands/outputs/vtss_hygr.tif
task failed
 outpath exists: True
1
C:/Workspace/VTWetlands/VTWetlands/outputs/lclu16_wet.tif
task failed
 outpath exists: True
2
C:/Workspace/VTWetlands/VTWetlands/outputs/lclu16_shr.tif
task failed
 outpath exists: True
3
C:/Workspace/VTWetlands/VTWetlands/outputs/lclu16_agr.tif
task failed
 outpath exists: True
4
C:/Workspace/VTWetlands/VTWetlands/outputs/rcpp_score.tif
task failed
 outpath exists: True
5
C:/Workspace/VTWetlands/VTWetlands/outputs/vswi_class.tif
task failed
 outpath exists: False
6
C:/Workspace/VTWetlands/VTWetlands/outputs/vswi_advis.tif
task failed
 outpath exists: False


In [115]:
# extract values from list of rasters at points
def arcpy_globExtractMultiValues2Points(inpath,globsearch,inpoints):
    os.chdir(inpath)
    inrasters = glob.glob(globsearch)
    print(inrasters)
    arcpy.sa.ExtractMultiValuesToPoints(inpoints,inrasters)
    return()
inpoints = outfolder + outname
arcpy_globExtractMultiValues2Points(inpath="D:/GeoData/Cropscape_VT",
                                    globsearch="*.tif",
                                    inpoints=inpoints)
arcpy_globExtractMultiValues2Points(inpath='D:\\GeoData\\LandLandcov_BaseLC2016\\',
                                    globsearch="*.tif",
                                    inpoints=inpoints)
os.chdir(wd)
arcpy_globExtractMultiValues2Points(inpath=outfolder,
                                    globsearch="*.tif",
                                    inpoints=inpoints)

['CDL_2008_clip_20201002155541_287304339.tif', 'CDL_2009_clip_20201002155541_287304339.tif', 'CDL_2010_clip_20201002155541_287304339.tif', 'CDL_2011_clip_20201002155541_287304339.tif', 'CDL_2012_clip_20201002155541_287304339.tif', 'CDL_2013_clip_20201002155541_287304339.tif', 'CDL_2014_clip_20201002155541_287304339.tif', 'CDL_2015_clip_20201002155541_287304339.tif', 'CDL_2016_clip_20201002155541_287304339.tif', 'CDL_2017_clip_20201002155541_287304339.tif', 'CDL_2018_clip_20201002155541_287304339.tif', 'CDL_2019_clip_20201002155541_287304339.tif']
['LandLandcov_BaseLC2016.tif']
['lclu16_agr.tif', 'lclu16_shr.tif', 'lclu16_wet.tif', 'rcpp_score.tif', 'vtss_hygr.tif']


()

In [114]:
# remove overlapping points
https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/select-layer-by-location.htm

C:/Workspace/VTWetlands/VTWetlands/outputs/randompoints.shp
